# Split Samples
## Split the subjects' into contiguous groups based on WCST performance


---
```
author:             Zach Wolpe
email:              zach.wolpe@medibio.com.au
date:               05 June 2022
```
---

In [2]:
import sys
sys.path.append('../process data')
from dependencies import *

In [3]:
# load dataframes -----------------------------++
data_path       = '../data objects/final_dataframes'
covariates      = pd.read_csv(f'{data_path}/covariates.csv')
wcst_raw_data   = pd.read_csv(f'{data_path}/wcst_raw_data.csv')

covariates.head()

,participant,nback_status,nback_reaction_time_ms,fitts_mean_deviation,corsi_block_span,wcst_accuracy,wcst_RT,demographics_age_a,demographics_gender_a,demographics_handedness_a,demographics_education_a,demographics_income_a,demographics_computer_hours_a,demographics_age_group,demographics_mean_reation_time_ms,navon_perc_correct,navon_reaction_time_ms,global_local
0,100934.0,0.900,2225.9125,-5.70,4.0,0.84,1795.16,28.0,male,right,university,7.0,25.0,25-34,11453.571429,0.314103,631.176282,-0.294872
1,103322.0,0.925,2373.6250,-203.05,6.0,0.87,1406.27,51.0,male,right,university,6.0,82.0,45-54,5294.428571,0.919872,979.759615,0.006410
2,107700.0,0.950,2138.7750,-342.65,6.0,0.83,1415.35,35.0,male,right,university,4.0,4.0,25-34,4175.714286,0.919872,710.958333,-0.006410
3,117200.0,0.775,2403.7500,-85.05,4.0,0.79,1154.50,39.0,male,right,university,3.0,34.0,35-44,3637.142857,0.291667,691.000000,0.083333
4,117306.0,0.975,2343.2625,-141.80,5.0,0.83,2044.93,28.0,male,right,university,3.0,5.0,25-34,7471.142857,0.762821,1290.333333,-0.141026


In [4]:
len(covariates.participant.unique())


fig = go.Figure()
fig.add_trace(go.Histogram(x=covariates.wcst_accuracy, marker_color='steelblue'))
fig.update_layout(template='none', height=450, width=450)
fig.show()


In [5]:
# split into n equal groups
n = 5
s = len(covariates.participant.unique())
print('n participants: ', s)

# n per group

g1 = s//n
g0 = s-g1*(n-1)
nn = [g0]
[nn.append(g1) for i in range(n-1)]


nn_ = [0]
[nn_.append(i) for i in nn]
nn_ = np.cumsum(nn_)
print('grouping indices ')
nn_


n participants:  214


array([  0,  46,  88, 130, 172, 214])

In [26]:
covar_sort = covariates[['participant','wcst_accuracy']].copy().sort_values('wcst_accuracy')
covar_sort = covar_sort.reset_index(drop=True)
covar_sort['RL_group'] = 0

for n in range(len(nn_)-1): covar_sort.loc[(nn_[n] < covar_sort.index) & (covar_sort.index <= nn_[n+1]),['RL_group']] = n
covar_sort.head(3)

,participant,wcst_accuracy,RL_group
0,247707.0,0.41,0
1,557453.0,0.42,0
2,726906.0,0.42,0


In [38]:

# save indices
store = f'{data_path}/sort_covariates.csv'
try:
    covar_sort.to_csv(store, index=False)
except:
    print(f'file aready exists at path: {store}')
